此示例显示可以组合因子以创建新因子，可以使用`mask`参数对因子进行排名，以及如何使用`pipeline`API创建多头和空头列表。

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-1-1 --end 2018-5-2
from logbook import Logger
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

log = Logger('回测')

def initialize(context):

    sma_short = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)
    sma_long = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=100)

    # Combined factors to create new factors
    sma_quotient = sma_short / sma_long

    # Create a screen to remove penny stocks
    remove_penny_stocks = sma_short > 1.0

    sma_rank = sma_quotient.rank(mask=remove_penny_stocks)

    # Rank a factor using a mask to ignore the values we're
    # filtering out by passing mask=remove_penny_stocks to rank.
    longs = sma_rank.top(200, mask=remove_penny_stocks)
    shorts = sma_rank.bottom(200, mask=remove_penny_stocks)

    pipe_columns = {'longs': longs, 'shorts': shorts, 'sma_rank': sma_rank}

    # Use multiple screens to narrow the universe
    pipe_screen = (longs | shorts)

    pipe = Pipeline(columns=pipe_columns, screen=pipe_screen)
    attach_pipeline(pipe, 'example')


def before_trading_start(context, data):
    context.output = pipeline_output('example')

    context.longs = context.output[context.output.longs]
    context.shorts = context.output[context.output.shorts]

    context.security_list = context.shorts.index.union(context.longs.index)

    # Print the 5 securities with the lowest sma_rank.
    print("SHORT LIST")
    log.info(
        "\n" + str(context.shorts.sort_values(['sma_rank'], ascending=True).head()))

    # Print the 5 securities with the highest sma_rank.
    print("LONG LIST")
    log.info(
        "\n" + str(context.longs.sort_values(['sma_rank'], ascending=False).head()))

SHORT LIST
[2018-05-03 01:40:35.558927] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
思美传媒(002712)  False    True       3.0
天润曲轴(002283)  False    True       4.0
金石东方(300434)  False    True       5.0
LONG LIST
[2018-05-03 01:40:35.564423] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3026.0
上海凤凰(600679)   True   False    3025.0
齐翔腾达(002408)   True   False    3024.0
卓郎智能(600545)   True   False    3023.0
南洋科技(002389)   True   False    3022.0
SHORT LIST
[2018-05-03 01:40:35.578241] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
天润曲轴(002283)  False    True       3.0
金石东方(300434)  False    True       4.0
思美传媒(002712)  False    True       5.0
LONG LIST
[2018-05-03 01:40:35.587655] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3028.0
上海凤凰(600679)   True   False    3027.0


SHORT LIST
[2018-05-03 01:40:36.509586] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
LONG LIST
[2018-05-03 01:40:36.517347] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3069.0
江苏国信(002608)   True   False    3068.0
中航沈飞(600760)   True   False    3067.0
贝瑞基因(000710)   True   False    3066.0
柘中股份(002346)   True   False    3065.0
SHORT LIST
[2018-05-03 01:40:36.526579] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
LONG LIST
[2018-05-03 01:40:36.531172] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3071.0
江苏国信(002608)   True   False    3070.0


SHORT LIST
[2018-05-03 01:40:36.762451] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
科大国创(300520)  False    True       3.0
世名科技(300522)  False    True       4.0
微光股份(002801)  False    True       5.0
LONG LIST
[2018-05-03 01:40:36.768855] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3108.0
江苏国信(002608)   True   False    3107.0
ST云维(600725)   True   False    3106.0
贝瑞基因(000710)   True   False    3105.0
柘中股份(002346)   True   False    3104.0
SHORT LIST
[2018-05-03 01:40:36.780175] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
科大国创(300520)  False    True       3.0
世名科技(300522)  False    True       4.0
微光股份(002801)  False    True       5.0
LONG LIST
[2018-05-03 01:40:36.785746] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3109.0
江苏国信(002608)   True   False    3108.0


SHORT LIST
[2018-05-03 01:40:37.039479] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
世名科技(300522)  False    True       3.0
中科创达(300496)  False    True       4.0
华媒控股(000607)  False    True       5.0
LONG LIST
[2018-05-03 01:40:37.048190] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3127.0
江苏国信(002608)   True   False    3126.0
天山股份(000877)   True   False    3125.0
湖南天雁(600698)   True   False    3124.0
同力水泥(000885)   True   False    3123.0
SHORT LIST
[2018-05-03 01:40:37.059173] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
世名科技(300522)  False    True       3.0
华媒控股(000607)  False    True       4.0
中科创达(300496)  False    True       5.0
LONG LIST
[2018-05-03 01:40:37.062978] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3130.0
江苏国信(002608)   True   False    3129.0


SHORT LIST
[2018-05-03 01:40:37.330014] INFO: 回测: 
               longs  shorts  sma_rank
金冠电气(300510)   False    True       1.0
四通股份(603838)   False    True       2.0
华媒控股(000607)   False    True       3.0
开尔新材(300234)   False    True       4.0
*ST河化(000953)  False    True       5.0
LONG LIST
[2018-05-03 01:40:37.336553] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3161.0
天山股份(000877)   True   False    3160.0
至纯科技(603690)   True   False    3159.0
江苏国信(002608)   True   False    3158.0
江阴银行(002807)   True   False    3157.0
SHORT LIST
[2018-05-03 01:40:37.350347] INFO: 回测: 
               longs  shorts  sma_rank
金冠电气(300510)   False    True       1.0
四通股份(603838)   False    True       2.0
华媒控股(000607)   False    True       3.0
开尔新材(300234)   False    True       4.0
*ST河化(000953)  False    True       5.0
LONG LIST
[2018-05-03 01:40:37.356744] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3164.0
天山股份(000877)   True   Fals

...

SHORT LIST
[2018-05-03 01:40:42.167648] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
宣亚国际(300612)   False    True       2.0
赛摩电气(300466)   False    True       3.0
ST云维(600725)   False    True       4.0
*ST众和(002070)  False    True       5.0
LONG LIST
[2018-05-03 01:40:42.175617] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3480.0
重庆钢铁(601005)   True   False    3479.0
贵州燃气(600903)   True   False    3478.0
泰禾集团(000732)   True   False    3477.0
新城控股(601155)   True   False    3476.0
SHORT LIST
[2018-05-03 01:40:42.192363] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
宣亚国际(300612)   False    True       2.0
赛摩电气(300466)   False    True       3.0
ST云维(600725)   False    True       4.0
*ST信通(600289)  False    True       5.0
LONG LIST
[2018-05-03 01:40:42.198881] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3481.0
重庆钢铁(601005)   True   Fals

SHORT LIST
[2018-05-03 01:40:42.448278] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
ST保千里(600074)  False    True       2.0
*ST信通(600289)  False    True       3.0
乐视网(300104)    False    True       4.0
顺威股份(002676)   False    True       5.0
LONG LIST
[2018-05-03 01:40:42.454064] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3493.0
泰禾集团(000732)   True   False    3492.0
重庆钢铁(601005)   True   False    3491.0
三六零(601360)    True   False    3490.0
华森制药(002907)   True   False    3489.0
SHORT LIST
[2018-05-03 01:40:42.463976] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
ST保千里(600074)  False    True       2.0
乐视网(300104)    False    True       3.0
*ST信通(600289)  False    True       4.0
顺威股份(002676)   False    True       5.0
LONG LIST
[2018-05-03 01:40:42.470526] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3493.0
泰禾集团(000732)   True   Fals

SHORT LIST
[2018-05-03 01:40:42.729896] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
大连电瓷(002606)   False    True       3.0
*ST信通(600289)  False    True       4.0
ST巴士(002188)   False    True       5.0
LONG LIST
[2018-05-03 01:40:42.736120] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3494.0
重庆钢铁(601005)   True   False    3493.0
上海钢联(300226)   True   False    3492.0
用友网络(600588)   True   False    3491.0
华森制药(002907)   True   False    3490.0
SHORT LIST
[2018-05-03 01:40:42.746580] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
大连电瓷(002606)   False    True       3.0
*ST信通(600289)  False    True       4.0
ST巴士(002188)   False    True       5.0
LONG LIST
[2018-05-03 01:40:42.752952] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3495.0
重庆钢铁(601005)   True   Fals

SHORT LIST
[2018-05-03 01:40:43.016556] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
ST巴士(002188)   False    True       3.0
*ST信通(600289)  False    True       4.0
华仁药业(300110)   False    True       5.0
LONG LIST
[2018-05-03 01:40:43.023347] INFO: 回测: 
              longs  shorts  sma_rank
万兴科技(300624)   True   False    3503.0
贵州燃气(600903)   True   False    3502.0
合肥城建(002208)   True   False    3501.0
盘龙药业(002864)   True   False    3500.0
用友网络(600588)   True   False    3499.0
SHORT LIST
[2018-05-03 01:40:43.038667] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
ST巴士(002188)   False    True       3.0
*ST信通(600289)  False    True       4.0
华仁药业(300110)   False    True       5.0
LONG LIST
[2018-05-03 01:40:43.045072] INFO: 回测: 
              longs  shorts  sma_rank
万兴科技(300624)   True   False    3503.0
合肥城建(002208)   True   Fals

KeyError: 'sma_rank'

**运行时间8.81秒**